# Example 1 - Plot power spectra

`PyCurious` offers convenience functions to extract the radial power spectrum. This has been covered in the Bouligand folder of jupyter notebooks and is largely duplicated here.

In this notebook we plot the radial power spectrum using the analytical expression, and the spectrum computed from a synthetic magnetic anomaly. (This can be generated from `Bouligand_forward.py` in the `tests` directory.)

### Contents

- [Radial power spectrum](#Power-spectrum-from-FFT)
- [Azimuthal power spectrum](#Azimuthal-power-spectrum)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import pycurious

## Radial power spectrum

The radial power spectrum is computed from a square window of the magnetic anomaly. Methods to select window sizes and compute the Fast Fourier Transform (FFT) belong to the `CurieGrid` object.

`CurieGrid` achieves the following purposes:

- Upward continuation
- Reduction to the pole
- Compute the radial power spectrum using FFT

The shape of the radial power spectrum is heavily dependent on window size. Resolution of long wavelength features require large windows.

> Suggestion: use a window size **> 4 times** the maximum Curie depth.


In [ ]:
# load magnetic anomaly - i.e. random fractal noise
mag_data = np.loadtxt("../../data/test_mag_data.txt")

nx, ny = 305, 305

x = mag_data[:,0]
y = mag_data[:,1]
d = mag_data[:,2].reshape(ny,nx)

xmin, xmax = x.min(), x.max()
ymin, ymax = y.min(), y.max()

In [ ]:
## Plot random fractal noise

fig = plt.figure()
ax1 = fig.add_subplot(111)
im1 = ax1.imshow(d)
fig.colorbar(im1, label='nT')

In [ ]:
# initialise CurieGrid object
grid = pycurious.CurieGrid(d, xmin, xmax, ymin, ymax)

# pick the centroid
xpt = 0.5*(xmin + xmax)
ypt = 0.5*(ymin + ymax)

window_size = 304e3
subgrid = grid.subgrid(window_size, xpt, ypt)

In [ ]:
# compute radial power spectrum
k, Phi, sigma_Phi = grid.radial_spectrum(subgrid)

# plot radial power spectrum
fig = plt.figure()
ax1 = fig.add_subplot(111, xlabel="wavenumber (rad/km)", ylabel="radial power spectrum")
ax1.plot(k, Phi, '-o')
plt.show()

**Choice of taper**

The default taper is the hanning filter (see [`numpy.hanning`](#hanning) for more details), but other functions can be passed to taper, or simply set it to `None`. There is a significant offset in the power spectrum with tapering functions, however, it is the *slope* of the spectrum with wavenumber that is most important when it comes to determining Curie depth.

In [ ]:

k1, Phi1, sigma_Phi2 = grid.radial_spectrum(subgrid, taper=None)
k2, Phi2, sigma_Phi2 = grid.radial_spectrum(subgrid, taper=np.hanning)
k3, Phi3, sigma_Phi2 = grid.radial_spectrum(subgrid, taper=np.hamming)


fig = plt.figure()
ax1 = fig.add_subplot(111, xlabel="wavenumber (rad/km)", ylabel="radial power spectrum")
ax1.plot(k1, Phi1, '-o', label='none')
ax1.plot(k2, Phi2, '-o', label='hanning')
ax1.plot(k3, Phi3, '-o', label='hamming')
ax1.legend()
plt.show()

## Azimuthal power spectrum

The azimuthal spectrum computes the FFT on a square window that is polarised at a range of radii. This is useful to explore linear trends in the magnetic anomaly that may align with a particular foliation or strike orientation.

```python
azimuthal_spectrum(subgrid, theta=5.0)
```

`theta` controls the thickness of each azimuth (in degrees).

In [ ]:
k, Phi, theta = grid.azimuthal_spectrum(subgrid, theta=20.0)

fig = plt.figure()
ax1 = fig.add_subplot(111, ylabel=r'azimuth $\theta$', xlabel='wavenumber (rad/km)')
im1 = ax1.pcolor(k, theta, Phi)
fig.colorbar(im1, label='radial power spectrum')

Plotting each azimuth on the power spectrum-wavenumber axis highlights the different in slope for different polarisations of the magnetic anomaly.

In [ ]:
fig = plt.figure()
ax1 = fig.add_subplot(111, xlabel="wavenumber (rad/km)", ylabel="radial power spectrum")

for i, angle in enumerate(theta):
    ax1.plot(k, Phi[i], label=r'$\theta = {}^\circ$'.format(angle))

ax1.legend(bbox_to_anchor=(1,1))
plt.show()